[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mparrott-at-wiris/aimodelshare/blob/master/notebooks/moral_compass_playground_setup.ipynb)


# Moral Compass Playground Setup

This notebook creates a new Model Playground and seeds it with a starter submission that includes Moral Compass metadata for fairness (key: `Moral_Compass_Fairness`, value: `"0"`). It then creates the Justice & Equity challenge moral compass table associated with the playground and performs an initial sync of a minimal metric so that the challenge leaderboard is initialized.

The notebook is designed to be resilient: if credentials or tokens are not present it will skip protected operations gracefully so it can still be opened and inspected.


## 1. Install Dependencies

In [ ]:
%pip install -q aimodelshare scikit-learn pandas numpy seaborn

## 2. Imports & Version Info

In [ ]:
import os, json, time, logging
import numpy as np, pandas as pd, seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

import aimodelshare as ai
from aimodelshare.playground import ModelPlayground
from aimodelshare.moral_compass import MoralcompassApiClient, ChallengeManager
from aimodelshare.moral_compass.challenge import JusticeAndEquityChallenge

print('aimodelshare version:', ai.__version__)
logging.basicConfig(level=logging.INFO)


## 3. Configuration
Set the playground identifier. Modify `PLAYGROUND_ID` if you need a unique name.

In [ ]:
# You can change this to create a fresh playground
PLAYGROUND_ID = 'moral_compass_quickstart'  # keep lowercase, hyphens/underscores only
PLAYGROUND_PRIVATE = True  # set False to make public (if your credentials allow)
TABLE_SUFFIX = '-mc'  # moral compass table naming suffix
USERNAME = os.getenv('AIMODELSHARE_USERNAME') or os.getenv('username') or 'demo_user'
PLAYGROUND_URL_PLACEHOLDER = f'https://example.com/playground/{PLAYGROUND_ID}'  # used for table ownership metadata
print('Configured PLAYGROUND_ID =', PLAYGROUND_ID)


## 4. Credential & Token Setup (Best Effort)
The library can source tokens automatically from environment variables (e.g. `JWT_AUTHORIZATION_TOKEN`, or legacy AWS tokens). This cell attempts a lightweight configuration; failures are non-fatal.

In [ ]:
try:
    from aimodelshare.aws import get_aws_token
    token = get_aws_token()
    if token:
        os.environ['AWS_TOKEN'] = token
        print('AWS token acquired.')
except Exception as e:
    print('Could not auto-acquire AWS token (ok for demo):', e)

try:
    from aimodelshare.modeluser import get_jwt_token
    if os.getenv('AIMODELSHARE_USERNAME') and os.getenv('AIMODELSHARE_PASSWORD'):
        get_jwt_token(os.getenv('AIMODELSHARE_USERNAME'), os.getenv('AIMODELSHARE_PASSWORD'))
        print('JWT token retrieved.')
    else:
        print('No username/password in env; skipping JWT retrieval.')
except Exception as e:
    print('JWT retrieval skipped/failed (ok for demo):', e)


## 5. Create / Load Playground
If the playground already exists, creation will typically raise a handled exception or simply attach to the existing resource.

In [ ]:
playground = ModelPlayground(input_type='tabular', task_type='classification', private=PLAYGROUND_PRIVATE)
try:
    # Dummy labels to satisfy create() minimal requirement (can be empty list)
    playground.create(eval_data=[], public=not PLAYGROUND_PRIVATE)
    print('Playground created.')
except Exception as e:
    print('Playground may already exist or creation failed gracefully:', e)


## 6. Prepare Simple Dataset & Baseline Model
Using seaborn's penguins dataset for a quick tabular classification example (predicting `sex`).

In [ ]:
penguins = sns.load_dataset('penguins').dropna()
X = penguins[['bill_length_mm','bill_depth_mm','flipper_length_mm','body_mass_g']]
y = penguins['sex']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
pipeline = Pipeline([('scaler', StandardScaler()), ('clf', LogisticRegression(max_iter=300))])
pipeline.fit(X_train, y_train)
preds = pipeline.predict(X_test)
print('Baseline model trained. Sample predictions:', preds[:5])


## 7. Submit Starter Model with Moral Compass Metadata
Attach a metadata key `Moral_Compass_Fairness` with value `'0'` so that a future fairness metric can be updated.

In [ ]:
submission_metadata = {
    'description': 'Starter baseline with fairness placeholder',
    'tags': 'moral_compass,baseline',
    'Moral_Compass_Fairness': '0'
}
try:
    playground.submit_model(
        model=pipeline,
        preprocessor=None,  # pipeline encapsulates preprocessing
        prediction_submission=preds,
        input_dict=submission_metadata,
        submission_type='experiment'
    )
    print('Starter model submitted with fairness metadata.')
except Exception as e:
    print('Model submission skipped/failed (possibly due to missing creds):', e)


## 8. View Leaderboard (If Available)

In [ ]:
try:
    lb = playground.get_leaderboard()
    import pandas as pd
    if isinstance(lb, dict):
        df = pd.DataFrame(lb)
    else:
        df = lb
    print(df.head())
except Exception as e:
    print('Could not fetch leaderboard (ok for demo):', e)


## 9. Create Justice & Equity Moral Compass Table
We now create (or ensure existence of) the challenge moral compass table following the naming convention `<playgroundId>-mc`.

In [ ]:
table_id = f'{PLAYGROUND_ID}{TABLE_SUFFIX}'
api_client = MoralcompassApiClient()
try:
    api_client.create_table(
        table_id=table_id,
        display_name=f'Moral Compass - {PLAYGROUND_ID}',
        playground_url=PLAYGROUND_URL_PLACEHOLDER
    )
    print('Challenge table created:', table_id)
except Exception as e:
    print('Table may already exist or creation failed gracefully:', e)

# Minimal initial sync: set a placeholder accuracy metric so leaderboard initializes
try:
    resp = api_client.update_moral_compass(
        table_id=table_id,
        username=USERNAME,
        metrics={'accuracy': 0.01},
        tasks_completed=0, total_tasks=6,
        questions_correct=0, total_questions=6  # using small placeholder counts
    )
    print('Initial moral compass sync response:', resp)
except Exception as e:
    print('Initial sync skipped/failed:', e)


## 10. Next Steps
You can now proceed to the second notebook: `moral_compass_model_submissions_and_challenge_progress.ipynb` to:
- Configure user credentials (if not already).
- Submit multiple model types (sklearn, Keras, PyTorch).
- Update fairness / accuracy metrics progressively via the Moral Compass API.

---
End of setup notebook.
